In [35]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip install -q pypdf
!pip install tqdm
!pip install -U langchain-community

In [36]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter #divisiones d texto
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS # buscador de similitud
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
import torch # las otras librerias pueden estarlo llamando
from langchain import PromptTemplate, LLMChain

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    pipeline,
    AutoModelForCausalLM
    )

In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #cuda permite correr los algoitmos para que se demoren menos
print(device) # Muestra en pantalla qué tipo de dispositivo se usará (cuda o cpu)


cpu


In [38]:
!mkdir -p ./pneumonia_docs
!cp "/content/drive/MyDrive/repaso/manual_neumo_nm-180-193.pdf" ./pneumonia_docs/IMSS_Guia_Neumonia.pdf
!ls ./pneumonia_docs

IMSS_Guia_Neumonia.pdf


In [39]:
import os  #  para manejar rutas y archivos del sistema operativo
path_docs = "./pneumonia_docs" # Define la ruta donde se guardaron los documentos PDF

os.listdir(path_docs) # Lista los archivos dentro de la carpeta especificada y los muestra por pantalla

['IMSS_Guia_Neumonia.pdf']

In [40]:
# transformacion de los documentos (leerlos y prepararlos para el modelo)
from tqdm.notebook import tqdm
from langchain.document_loaders import PyPDFLoader #cargar y leer archivos PDF.
from langchain.text_splitter import RecursiveCharacterTextSplitter #divisiones d texto


# Parámetros de división
chunk_size = 700 # longitud máxima de cada fragmento de texto en caracteres.
chunk_overlap = 100 # chunk coge los limites de contexto

# Crear lista de fragmentos
all_docs = []
text_splitter = RecursiveCharacterTextSplitter(  #divide el texto en fragmentos de tamaño determinado.
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
    ) # llena la lista con los fragmentos de texto

# Leer y dividir cada PDF
for name in tqdm(os.listdir(path_docs)):#recorrer todos los archivos pdf
    if name.endswith(".pdf"): #filtro adicicional para tomar solo archivos que finalicen en .pdf
        path_tmp = os.path.join(path_docs, name) #construcción de la ruta completa
        loader = PyPDFLoader(path_tmp) #carga el archivo pdf y lealo
        text = loader.load() #extracción del texto del pdf
        chunks_of_text = text_splitter.split_documents(text) # aplicación de división por fragmentos
        all_docs.extend(chunks_of_text) #agrega los fragmentos a la lista

print(f" Se generaron {len(all_docs)} fragmentos de texto.")

  0%|          | 0/1 [00:00<?, ?it/s]

 Se generaron 78 fragmentos de texto.


**modelos de embedding**

In [41]:
from pprint import pprint

def print_lines(text, max_ch=50): #max_ch establece el máximo de caracteres por línea
    current_line = ""  # Acumula las palabras de la línea actual
    for word in text.split():  # Recorre el texto palabra por palabra
        if len(current_line) + len(word) + 1 > max_ch:  # Si agregar la siguiente palabra supera el límite de caracteres, imprime la línea actual
            print(current_line)
            current_line = word  # Inicia una nueva línea con la palabra actual
        else:
            current_line += " " + word  # Si no se supera el límite, agrega la palabra a la línea con un espacio
    if current_line:
        print(current_line)

chunk = all_docs[0]
print_lines(chunk.page_content)
pprint(chunk.metadata)

 RESUMEN La neumonía es muy frecuente en la
práctica clínica por su frecuencia y elevada
morbimortalidad. Se clasifican, en función del
huésped, en inmunocompetente o in- munodeprimido
y, según el ámbito de ad- quisición, comunitarias
o nosocomiales. Las neumonías comunitarias se
subclasifican, según factores de riesgo y
gravedad, en fun- ción del lugar de tratamiento,
en ambula- torio, ingresado o ingresado en
unidades de cuidados intensivos. Las neumonías no-
socomiales se clasifican en precoces (me- nos de 5
días) o tardías. El tratamiento se inicia
generalmente de forma empírica de forma lo más
precoz posible y con mono- terapia o combinaciones
según el tipo de
{'creationdate': '',
 'creator': 'PyPDF',
 'moddate': '2025-10-13T00:36:08+00:00',
 'page': 0,
 'page_label': '1',
 'producer': 'iLovePDF',
 'source': './pneumonia_docs/IMSS_Guia_Neumonia.pdf',
 'total_pages': 14}


In [43]:
def get_embeddings_model(model_path=None):
  # Modelos de embeddings
  model1 = "sentence-transformers/all-MiniLM-l6-v2"
  model2 = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
  model3 = "intfloat/multilingual-e5-base"

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"device: {device}")

  model_kwargs = {'device': device} # Define en qué dispositivo se ejecutará
  encode_kwargs = {'normalize_embeddings': False}  # Indica que no se normalicen los embeddings generados


  embeddings1 = HuggingFaceEmbeddings(
      model_name=model1,
      model_kwargs=model_kwargs,
      encode_kwargs=encode_kwargs
  )

  embeddings2 = HuggingFaceEmbeddings(
      model_name=model2,
      model_kwargs=model_kwargs,
      encode_kwargs=encode_kwargs
  )

  embeddings3 = HuggingFaceEmbeddings(
      model_name=model3,
      model_kwargs=model_kwargs,
      encode_kwargs=encode_kwargs
  )

  print(" Modelos cargados correctamente:")
  print("-", model1)
  print("-", model2)
  print("-", model3)

  return embeddings1, embeddings2, embeddings3

**Modelos seleccionados**

Se eligieron tres modelos de embeddings: sentence-transformers/all-MiniLM-l6-v2, sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 y intfloat/multilingual-e5-base, porque permiten evaluar diferentes combinaciones de rendimiento, soporte para el idioma español y precisión semántica. El primero es un modelo base rápido y liviano, ideal para pruebas iniciales; el segundo añade comprensión multilingüe, lo que mejora su desempeño en textos en español; y el tercero, aunque más pesado, ofrece una representación semántica más precisa y fue el que mejor respondió en las pruebas con preguntas clínicas sobre neumonía.

Se desecharon otros modelos como distiluse-base-multilingual-cased-v2, all-mpnet-base-v2, sentence-t5-base y multi-qa-MiniLM-L6-cos-v1, ya que o bien están optimizados solo para inglés, no ofrecen mejoras significativas en comprensión en español, o requieren mayores recursos computacionales, lo que los hace poco prácticos para un sistema de recuperación ligera. Además, se descartaron versiones más grandes de la familia E5 y MiniLM (como intfloat/multilingual-e5-large o all-MiniLM-L12-v2) porque su tamaño aumentaba considerablemente el tiempo de respuesta y el consumo de memoria sin un incremento proporcional en la precisión. Por tanto, se mantuvieron los tres modelos seleccionados por su equilibrio entre desempeño, compatibilidad con español y eficiencia computacional.

In [44]:
embeddings1, embeddings2, embeddings3 = get_embeddings_model()

device: cpu
 Modelos cargados correctamente:
- sentence-transformers/all-MiniLM-l6-v2
- sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
- intfloat/multilingual-e5-base


In [9]:
text = "La neumonía es una infección que afecta los pulmones." # Texto de prueba que se convertirá en un vector numérico (embedding)

modelos = [embeddings1, embeddings2, embeddings3] # Lista con los tres modelos de embeddings cargados anteriormente

contador = 1

# Recorre cada modelo de la lista y genera su embedding del texto
for modelo in modelos:
  # Convierte el texto en un vector (embedding) usando el modelo actual
    resultado = modelo.embed_query(text)
    print("El modelo", contador, "tiene", len(resultado), "dimensiones")
    contador = contador + 1  # Aumenta el contador para pasar al siguiente model

El modelo 1 tiene 384 dimensiones
El modelo 2 tiene 384 dimensiones
El modelo 3 tiene 768 dimensiones


In [45]:
# Crea tres bases vectoriales (vectorstores) usando FAISS,
# una por cada modelo de embeddings para comparar su desempeño

db1 = FAISS.from_documents(all_docs, embeddings1)
db2 = FAISS.from_documents(all_docs, embeddings2)
db3 = FAISS.from_documents(all_docs, embeddings3)

In [46]:

pregunta = "¿sintomas de la neumonia?"

# Busca los 5 fragmentos más similares en cada base vectorial (una por modelo)
search1 = db1.similarity_search(pregunta, k=5)
search2 = db2.similarity_search(pregunta, k=5)
search3 = db3.similarity_search(pregunta, k=5)

# Muestra el fragmento más relevante encontrado por cada modelo
print(" Resultado con el modelo 1:")
print_lines(search1[0].page_content)

print("\n Resultado con el modelo 2:")
print_lines(search2[0].page_content)

print("\n Resultado con el modelo 3:")
print_lines(search3[0].page_content)

 Resultado con el modelo 1:
 La neumonía con mala evolución puede deberse a
las siguientes causas: • Enfermedad no infecciosa:
principal- mente tromboembolismo pulmonar, ne-
oplasia, bronquiectasias o insuficiencia Neumonía
185

 Resultado con el modelo 2:
 forma lo más precoz posible y con mono- terapia o
combinaciones según el tipo de neumonía y la
clasificación previa. En la neumonía comunitaria
el S. pneumoniae es el patógeno más frecuente
aunque la co- bertura de patógenos atípicos se
asocia a menor mortalidad en estudios amplios. Si
se inicia de forma intravenosa la terapia se-
cuencial a partir del segundo día es segura si se
ha alcanzado la estabilidad clínica. En caso de
mala evolución hay que considerar diagnósticos
alternativos infecciosos y no infecciosos,
reevaluar microbiológicamen- te, descartar
complicaciones y considerar modificar la
antibioterapia para ampliar el espectro. En la
neumonía nosocomial, un

 Resultado con el modelo 3:
 neumonías de origen vírico, se ident

In [47]:
import re #libreria para limpiar texto

# Función que obtiene el contexto más relevante para una pregunta
def get_context(db, question, top_k=2):
    searchDocs = db.similarity_search(question, k=top_k)
    texto = "\n".join([x.page_content for x in searchDocs])
    texto_limpio = re.sub(r"\t+", " ", texto)
    return texto_limpio[:3000]


pregunta = "¿sintomas de la neumonia?"

# Obtiene los contextos más relevantes de cada modelo
contexto1 = get_context(db1, pregunta)
contexto2 = get_context(db2, pregunta)
contexto3 = get_context(db3, pregunta)


print(" Contexto del modelo 1:\n")
print(contexto1)

print("\n Contexto del modelo 2:\n")
print(contexto2)

print("\n Contexto del modelo 3:\n")
print(contexto3)

 Contexto del modelo 1:

La neumonía con mala evolución puede
deberse a las siguientes causas:
• Enfermedad no infecciosa: principal-
mente tromboembolismo pulmonar, ne-
oplasia, bronquiectasias o insuficiencia
Neumonía 185
Neumonía
C.J. Álvarez Martínez 1, Á. Casanova Espinosa 2, 
J.R. Donado Uña 3, Mª.A. Perpiñá Ferri 4
1Servico de Neumología. Hospital Universitario 12 de Octubre.
Madrid. 2Servico de Neumología Hospital del Henares. Madrid. 
3Servico de Neumología. Hospital de Fuenlabrada. Madrid.
4Servico de Neumología. Hospital Severo Ochoa. Madrid.
Capítulo 11

 Contexto del modelo 2:

forma lo más precoz posible y con mono-
terapia o combinaciones según el tipo de
neumonía y la clasificación previa. En la
neumonía comunitaria el S. pneumoniae es
el patógeno más frecuente aunque la co-
bertura de patógenos atípicos se asocia a
menor mortalidad en estudios amplios. Si
se inicia de forma intravenosa la terapia se-
cuencial a partir del segundo día es segura
si se ha alcanzado la est

Al evaluar los tres modelos de embeddings frente a la pregunta ¿síntomas de la neumonía?, se observó que el modelo 3 (intfloat/multilingual-e5-base) fue el que ofreció el mejor resultado, ya que recuperó un fragmento del texto donde se describen directamente las manifestaciones clínicas de la enfermedad: malestar general, fiebre, escalofríos, tos, expectoración y dolor torácico, entre otros. Este modelo mostró una mayor precisión semántica, identificando la sección del documento más relevante en función del significado de la pregunta.

Por el contrario, el modelo 1 (all-MiniLM-l6-v2), al estar entrenado principalmente en inglés y con menor capacidad semántica, recuperó un párrafo relacionado con la falta de respuesta terapéutica, sin conexión con los síntomas. El modelo 2 (paraphrase-multilingual-MiniLM-L12-v2), aunque multilingüe y compatible con español, priorizó información sobre tratamientos y clasificación de neumonías, lo que indica que comprendió el tema general, pero no la intención específica de la pregunta.

En conclusión, el modelo intfloat/multilingual-e5-base destacó por su mayor capacidad para relacionar términos médicos y contexto clínico, proporcionando la respuesta más completa y relevante sobre los síntomas de la neumonía, razón por la cual fue considerado el modelo más adecuado para este tipo de consultas médicas.

**LLM: modelos the question-answering**


In [49]:

import torch
from transformers import AutoModelForQuestionAnswering
from transformers import TFAutoModelForQuestionAnswering

Se seleccionaron tres modelos de lenguaje con el objetivo de evaluar el desempeño en tareas de pregunta–respuesta aplicadas a textos clínicos en español, considerando la influencia del idioma y la eficiencia computacional.

El modelo mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es se eligió por estar entrenado específicamente en español, lo que lo convertía en una referencia ideal para analizar el comportamiento de un modelo monolingüe en un contexto médico técnico. El modelo deepset/minilm-uncased-squad2, aunque entrenado en inglés, se incluyó por su alta eficiencia y generalización semántica, además de ser un modelo compacto (MiniLM) ampliamente probado en tareas de comprensión lectora. Su inclusión buscaba verificar si un modelo en inglés podía mantener un buen rendimiento con documentos en español, lo cual finalmente se comprobó. Por último, distilbert-base-multilingual-cased fue seleccionado por su capacidad multilingüe y bajo costo computacional, lo que permite ejecutar el sistema en entornos con recursos limitados y comparar su desempeño frente a modelos más grandes.

En conjunto, estos tres modelos permitieron contrastar precisión, idioma y eficiencia, cubriendo desde un modelo entrenado exclusivamente en español hasta uno multilingüe y otro en inglés de arquitectura liviana. Esta selección equilibrada garantizó una evaluación completa del rendimiento del sistema RAG en distintas condiciones lingüísticas y de procesamiento.

con mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es

In [50]:

question = "¿Qué es la neumonía?"
context = get_context(db3, question, top_k=3)


model_ckpt = "mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

inputs = tokenizer(question, context, return_tensors="pt")
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)


pipe(question=question, context=context, topk=3)

Some weights of the model checkpoint at mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/question_answering.py:339: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)


[{'score': 0.33334866166114807,
  'start': 255,
  'end': 283,
  'answer': 'lesión inflamatoria pulmonar'},
 {'score': 0.17167994379997253,
  'start': 251,
  'end': 283,
  'answer': 'una\nlesión inflamatoria pulmonar'},
 {'score': 0.00799021776765585,
  'start': 262,
  'end': 283,
  'answer': 'inflamatoria pulmonar'}]

In [15]:

question = "¿cual es el tratamiento para nac ambulatoria?"
context = get_context(db3, question, top_k=3)


model_ckpt = "mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

inputs = tokenizer(question, context, return_tensors="pt")
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)


pipe(question=question, context=context, topk=3)

Some weights of the model checkpoint at mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'score': 0.10653464496135712,
  'start': 215,
  'end': 233,
  'answer': 'Levofloxacino oral'},
 {'score': 0.018636928871273994,
  'start': 556,
  'end': 591,
  'answer': 'cefalosporina de tercera generación'},
 {'score': 0.011992162093520164,
  'start': 215,
  'end': 228,
  'answer': 'Levofloxacino'}]

In [16]:

question = "¿duración del tratamiento antibiótico para pacientes que no requieren ingreso?"
context = get_context(db3, question, top_k=3)


model_ckpt = "mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

inputs = tokenizer(question, context, return_tensors="pt")
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)


pipe(question=question, context=context, topk=3)

Some weights of the model checkpoint at mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'score': 0.7656724154949188,
  'start': 1242,
  'end': 1251,
  'answer': '7-10 días'},
 {'score': 0.6039672791957855,
  'start': 1236,
  'end': 1251,
  'answer': 'entre\n7-10 días'},
 {'score': 0.01831491058692336, 'start': 1242, 'end': 1246, 'answer': '7-10'}]

In [19]:

question = "¿tratamiento para nac que precisa ingreso en uci, del grupo III?"
context = get_context(db3, question, top_k=3)


model_ckpt = "mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

inputs = tokenizer(question, context, return_tensors="pt")
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)


pipe(question=question, context=context, topk=3)

Some weights of the model checkpoint at mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'score': 0.4371949974447489,
  'start': 1113,
  'end': 1148,
  'answer': 'Cefalosporina de tercera generación'},
 {'score': 0.09280361235141754,
  'start': 1111,
  'end': 1148,
  'answer': '- Cefalosporina de tercera generación'},
 {'score': 0.0744614175055176,
  'start': 1113,
  'end': 1126,
  'answer': 'Cefalosporina'}]

**modelo deepset/minilm-uncased-squad2**

In [20]:

question = "¿Qué es la neumonía?"
context = get_context(db3, question, top_k=9)


model_ckpt = "deepset/minilm-uncased-squad2"
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

inputs = tokenizer(question, context, return_tensors="pt")
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)


pipe(question=question, context=context, topk=3)

config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


[{'score': 1.6036356842378154,
  'start': 1478,
  'end': 1489,
  'answer': 'neumocócica'},
 {'score': 0.10130611062049866,
  'start': 251,
  'end': 274,
  'answer': 'una\nlesión inflamatoria'},
 {'score': 0.08316279777500313,
  'start': 1478,
  'end': 1514,
  'answer': 'neumocócica, neumonía\nestafilocócica'}]

In [21]:

question = "¿tratamiento para nac ambulatorio?"
context = get_context(db3, question, top_k=9)


model_ckpt = "deepset/minilm-uncased-squad2"
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

inputs = tokenizer(question, context, return_tensors="pt")
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)


pipe(question=question, context=context, topk=3)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'score': 0.007654170971363783,
  'start': 1314,
  'end': 1351,
  'answer': 'Tratamiento en situaciones especiales'},
 {'score': 0.004091440699994564,
  'start': 400,
  'end': 439,
  'answer': 'combinación be-\ntalactámico y macrólido'},
 {'score': 0.0038593256613239646,
  'start': 1314,
  'end': 1325,
  'answer': 'Tratamiento'}]

In [22]:

question = "¿duración del tratamiento antibiótico para pacientes que no requieren ingreso?"
context = get_context(db3, question, top_k=9)

model_ckpt = "deepset/minilm-uncased-squad2"
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

inputs = tokenizer(question, context, return_tensors="pt")
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)

pipe(question=question, context=context, topk=3)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'score': 0.015263861045241356,
  'start': 1204,
  'end': 1225,
  'answer': 'Habitualmente las NAC'},
 {'score': 0.013314570300281048,
  'start': 1222,
  'end': 1251,
  'answer': 'NAC se tratan entre\n7-10 días'},
 {'score': 0.01244890468660742,
  'start': 1236,
  'end': 1251,
  'answer': 'entre\n7-10 días'}]

In [23]:

question = "¿tratamiento para nac que precisa ingreso en uci, del grupo III?"
context = get_context(db3, question, top_k=9)

model_ckpt = "deepset/minilm-uncased-squad2"
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

inputs = tokenizer(question, context, return_tensors="pt")
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)

pipe(question=question, context=context, topk=3)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'score': 0.004478990827919915,
  'start': 2713,
  'end': 2735,
  'answer': 'resolución radiológica'},
 {'score': 0.0022981173169682734,
  'start': 2710,
  'end': 2735,
  'answer': 'La resolución radiológica'},
 {'score': 0.0022191458265297115,
  'start': 2713,
  'end': 2723,
  'answer': 'resolución'}]

se puede notar que este modelo trbaja mejor y logra resporder a esa parte rara

** modelo distilbert-base-multilingual-cased**

In [24]:

question = "¿Qué es la neumonía?"
context = get_context(db3, question, top_k=9)

model_ckpt = "distilbert-base-multilingual-cased"
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

inputs = tokenizer(question, context, return_tensors="pt")
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)

pipe(question=question, context=context, topk=3)

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (857 > 512). Running this sequence through the model will result in indexing errors
Device set to use cpu


[{'score': 5.2836796385236084e-05,
  'start': 2820,
  'end': 2854,
  'answer': 'Neumología. Hospital Universitario'},
 {'score': 3.241574995627161e-05,
  'start': 2708,
  'end': 2738,
  'answer': 'Neumonía\nC.J. Álvarez Martínez'},
 {'score': 3.224316242267378e-05,
  'start': 2808,
  'end': 2854,
  'answer': '1Servico de Neumología. Hospital Universitario'}]

In [25]:

question = "¿cual es el tratamiento para nac ambulatoria?"
context = get_context(db3, question, top_k=9)

model_ckpt = "distilbert-base-multilingual-cased"
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

inputs = tokenizer(question, context, return_tensors="pt")
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)

pipe(question=question, context=context, topk=3)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Token indices sequence length is longer than the specified maximum sequence length for this model (911 > 512). Running this sequence through the model will result in indexing errors
Device set to use cpu


[{'score': 0.0002394113671471132,
  'start': 1057,
  'end': 1068,
  'answer': 'vancomicina'},
 {'score': 0.0001662501072132727,
  'start': 2695,
  'end': 2728,
  'answer': 'g cada 12 horas i.v.) o linezolid'},
 {'score': 0.00015910212096059695,
  'start': 2644,
  'end': 2691,
  'answer': 'epidemiológicos, las resistencias\n- Vancomicina'}]

In [26]:

question = "¿duración del tratamiento antibiótico para pacientes que no requieren ingreso?"
context = get_context(db3, question, top_k=9)

model_ckpt = "distilbert-base-multilingual-cased"
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

inputs = tokenizer(question, context, return_tensors="pt")
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)

pipe(question=question, context=context, topk=3)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Token indices sequence length is longer than the specified maximum sequence length for this model (932 > 512). Running this sequence through the model will result in indexing errors
Device set to use cpu


[{'score': 0.00014705650391988456,
  'start': 2401,
  'end': 2419,
  'answer': '. Perpiñá Ferri188'},
 {'score': 9.38033263082616e-05,
  'start': 2381,
  'end': 2395,
  'answer': '.R. Donado Uña'},
 {'score': 5.320167838362977e-05,
  'start': 2836,
  'end': 2883,
  'answer': 'los casos puede ser suficiente la mono-\nterapia'}]

In [28]:

question = "¿tratamiento para nac que precisa ingreso en uci, del grupo III??"
context = get_context(db3, question, top_k=9)

model_ckpt = "distilbert-base-multilingual-cased"
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

inputs = tokenizer(question, context, return_tensors="pt")
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)

pipe(question=question, context=context, topk=3)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Token indices sequence length is longer than the specified maximum sequence length for this model (927 > 512). Running this sequence through the model will result in indexing errors
Device set to use cpu


[{'score': 9.339710595668294e-05,
  'start': 2485,
  'end': 2546,
  'answer': 'las 24-48 horas de iniciado el trata-\nmiento y estabilización'},
 {'score': 6.682220555376261e-05,
  'start': 2656,
  'end': 2708,
  'answer': 'las 48 ó 72 horas o antes en caso de em-\npeoramiento'},
 {'score': 5.0898001063615084e-05,
  'start': 2656,
  'end': 2695,
  'answer': 'las 48 ó 72 horas o antes en caso de em'}]

Al evaluar los tres modelos de lenguaje (LLM), se observó que el modelo mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es ofreció los mejores resultados, mostrando respuestas coherentes, completas y directamente relacionadas con las preguntas clínicas, gracias a su entrenamiento específico en español y su afinación con el conjunto SQuAD2-es(Esto obliga al modelo no solo a encontrar la respuesta correcta cuando existe, sino también a reconocer cuándo no hay información suficiente en el texto). En cambio, el segundo modelo, deepset/minilm-uncased-squad2, aunque eficiente y rápido, tiende a confundirse con el idioma, generando respuestas parciales o inexactas al procesar textos médicos en español debido a su entrenamiento exclusivo en inglés. Finalmente, el modelo distilbert-base-multilingual-cased presentó un desempeño notablemente inferior: sus respuestas fueron vagas, incompletas y, en ocasiones, incoherentes, lo que se explica por su menor capacidad de comprensión contextual y la simplificación inherente a su arquitectura liviana. En síntesis, el primer modelo fue seleccionado por su equilibrio óptimo entre precisión semántica, dominio del idioma y fiabilidad en la extracción de información médica.

**pregunta ingresada por el usuario:**

In [18]:

question = input("\ningrese una pregunta:").strip()
context = get_context(db3, question, top_k=3)


model_ckpt = "mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

inputs = tokenizer(question, context, return_tensors="pt")
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)


pipe(question=question, context=context, topk=3)


ingrese una pregunta¿cuales son los sintomas de la neumonia?


Some weights of the model checkpoint at mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'score': 0.02587028406560421,
  'start': 423,
  'end': 459,
  'answer': 'malestar ge-\nneral, anorexia, fiebre'},
 {'score': 0.019845647737383842,
  'start': 423,
  'end': 451,
  'answer': 'malestar ge-\nneral, anorexia'},
 {'score': 0.009345129132270813,
  'start': 423,
  'end': 441,
  'answer': 'malestar ge-\nneral'}]